In [ ]:
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 1. 데이터 선택 및 결측치 정리

# print(X_train.info())
# # print(X_train.describe())
# print(y_train.info())

col_del=['cust_id']
col_num=['총구매액', '최대구매액', '내점일수', '내점당구매건수','주말방문비율', '구매주기']
col_cat=['주구매상품', '주구매지점']
col_y=['gender']

from sklearn.preprocessing import LabelEncoder
for col in col_cat:
	le=LabelEncoder()
	le.fit(X_train[col])
	X_train[col]=le.transform(X_train[col])
	le.fit(X_test[col])
	X_test[col]=le.transform(X_test[col])

# 2. 데이터 스플릿 및 스케일링
from sklearn.model_selection import train_test_split

x_tr, x_val , y_tr, y_val = train_test_split(pd.concat([X_train[col_num],X_train[col_cat]],axis=1),y_train[col_y],test_size=0.2)

from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(x_tr[col_num])
x_tr[col_num]=scaler.transform(x_tr[col_num])
x_val[col_num]=scaler.transform(x_val[col_num])
X_test[col_num]=scaler.transform(X_test[col_num])

# 3. 모델링 및 평가
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(x_tr,y_tr.values.ravel())
pred=rf.predict(x_val)
proba=rf.predict_proba(x_val)
from sklearn.metrics import roc_auc_score

#score=roc_auc_score(y_val,pred)
#print(score)
scorep=roc_auc_score(y_val,proba[:,1])
print(scorep)

# 정답제출을 위한 proba
proba=rf.predict_proba(pd.concat([X_test[col_num],X_test[col_cat]],axis=1))[:,1]
# # 답안 제출 참고
# # 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
pd.DataFrame({'cust_id': X_test.cust_id, 'gender': proba}).to_csv('003000000.csv', index=False)
p=pd.read_csv('003000000.csv')
print(p)
